In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import Geodesics as gd
import Model as md
import cmath as cmt
import glob
import os

In [ ]:
##Set simulation parameters

p = 5; q = 4; nlayers = 4 #Lattice parameters
nDecorr = 100 #Steps to decorrelate
nMeasurements = 50000 #Number of measurements
saveFreq = 1000 #Save frequency
#Set up model
model = md.FractonModel(p,q,nlayers)

# Define the directory where you want to save the files
save_dir = "../Data/P"+str(p)+"Q"+str(q)+"/"+str(nlayers)+"Layers"

# Check if the directory exists, if not, create it
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
#Set up model
model = md.FractonModel(p,q,nlayers)

#Number of spins in the border
nBorder = len(model.border)

#Initialize correlation and states arrays
corr = np.zeros((saveFreq,int(len(model.border)/2)))
accum_spins = np.zeros((saveFreq,len(model.lattice)))

#Initialize save counter
saveCounter = 0

#Simulation main loop
for iteration in range(nMeasurements):
    model.decorrelate(nDecorr)
    print(iteration)
    accum_spins[saveCounter,:] = model.spins
    corr[saveCounter,:] = model.getBorderCorrelations()/(nBorder)
    saveCounter += 1
    if(saveCounter == saveFreq):
        np.save(os.path.join(save_dir, "States" + str(int(iteration/saveFreq)) + ".npy"),np.array(accum_spins))
        np.save(os.path.join(save_dir, "Correlations" + str(int(iteration/saveFreq)) + ".npy"),corr)
        corr = np.zeros((saveFreq,int(len(model.border)/2)))
        accum_spins = np.zeros((saveFreq,len(model.lattice)))
        saveCounter = 0

del corr
del accum_spins

In [ ]:
corr_list = sorted(glob.glob("../Data/Correlations?.npy"))
corr_list += sorted(glob.glob("../Data/Correlations??.npy"))

states_list =sorted(glob.glob("../Data/States?.npy"))
states_list +=sorted(glob.glob("../Data/States??.npy"))

distances = np.linspace(1,int(nBorder/2),int(nBorder))

In [ ]:
allCorrelations = np.zeros((560,280))

for num, file in enumerate(states_list):
    states = np.load(file)
    print(num)
    for idx,state in enumerate(states):
        model.spins = state
        allCorrelations[:,:] += model.getBorderCorrelationsEachSpin()

In [ ]:
allCorrelations = np.load("../Data/MeansTest.npy")
allCorrelations = allCorrelations/1000000
distances = np.linspace(1,280,280)

In [ ]:
fig, ax = plt.subplots()
ax.plot(distances, np.sum(allCorrelations,axis = 0))
ax.set_ylabel(r'$\sum_{i=0}^{N_m}\sum_{n=0}^L C^i_n(d)/(L\cdot N)$')
ax.set_xlabel(r'$d$')
fig.savefig("/home/alejo/Desktop/BorderCorrelations.png", format = 'png',  pad_inches = 1,bbox_inches= 'tight',transparent=True)